In [18]:
import pandas as pd
# import string
import math
from rapidfuzz import process, fuzz

def split_route(route):
    route_clean = str(route).lower().strip()
    if ' to ' in route_clean:
        if ' via ' in route_clean:
            direct_route = route_clean.split(' via ')
            locations = direct_route[0].split(' to ')
        else:
            locations = route_clean.split(' to ')
        origin = locations[0].strip()
        destination = locations[-1].strip()
    else:
        origin = None
        destination = None

    # Extracting the first (departing) and last (arriving) locations
    return origin, destination

# Function to clean and get unique values from a DataFrame column
def get_unique_col_values(df, col_names):
    unique_vals = pd.concat([df[col].str.lower().str.strip() for col in col_names]).unique()
    return unique_vals
    
def closest_match(query, choices):
    try:
        closest_match = process.extractOne(query, choices, scorer=fuzz.WRatio)
        match = closest_match[0]
        score = closest_match[1]
        return match, score
    except Exception:
        return (None, 0)

def try_map(word,mapper):
    try:
        return mapper[word]
    except:
        return None

def calculate_distance(origin, destination):
    try:
        origin_lat, origin_lon = [float(x) for x in origin.split(', ')]
        destination_lat, destination_lon = [float(x) for x in destination.split(', ')]
        # Calculating haversine distance
        origin_lon, origin_lat, destination_lon, destination_lat = map(math.radians, [origin_lon, origin_lat, destination_lon, destination_lat])
        dlon = destination_lon - origin_lon
        dlat = destination_lat - origin_lat
        a = math.sin(dlat/2)**2 + math.cos(origin_lat) * math.cos(destination_lat) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        r = 6371
        return c * r
    except:
        return None

In [20]:
# Load the original dataset
df = pd.read_csv('data/Airline_review.csv').iloc[:, 1:]
df.dropna(subset='Route', inplace=True)

# Splitting Route into origin and destination
df['origin'], df['destination'] = zip(*df['Route'].apply(split_route))

In [ ]:
# Importing and cleaning the airport codes data
codes_df = pd.read_csv('data/airport_codes.csv', sep=';')
codes_df = codes_df.rename(columns={
    'Airport Code': 'code',
    'Airport Name': 'airport',
    'City Name': 'city'
}).apply(lambda x: x.str.lower().str.strip() if x.name in ['code', 'airport', 'city'] else x)
codes_df = codes_df[['code', 'airport', 'city', 'coordinates']]

In [ ]:
# Getting all unique airport codes, airport names and city names
unique_codes = get_unique_col_values(codes_df, ['code', 'airport', 'city'])

# Getting all unique origins and destinations
unique_originals = get_unique_col_values(df, ['origin', 'destination'])

# Finding the closest match to help merge dataframes
matches = []
scores = []
for word in unique_originals:
    match, score = closest_match(word,unique_codes)
    matches.append(match)
    scores.append(score)
score_df = pd.DataFrame({'word': unique_originals, 'match': matches, 'score': scores})

# Filtering out rows with match scores of less than 80
score_df = score_df.loc[score_df['score'] >= 80]

# Creating a dictionary to map coordinates onto matched values
melted_df = codes_df.melt(id_vars='coordinates', value_vars=['code', 'airport', 'city'], value_name='Key').drop('variable', axis=1)
result_dict = dict(zip(melted_df['Key'], melted_df['coordinates']))

# Adding coordinates to score_Df
score_df['coordinates'] = score_df['match'].apply(lambda x: try_map(x,result_dict))

In [ ]:
df_new = df[['Route', 'origin', 'destination', 'Overall_Rating', 'Recommended']].copy().dropna()

df_origins = pd.merge(df_new, score_df[['word','match','coordinates']], left_on= 'origin',right_on='word', how='left')
df_origins.rename(columns={'match':'origin_city','coordinates':'origin_coordinates'},inplace=True)
df_origins.drop(columns='word', inplace=True)

df_dest = pd.merge(df_origins, score_df[['word','match','coordinates']], left_on= 'destination',right_on='word', how='left')
df_dest.rename(columns={'match':'destination_city','coordinates':'destination_coordinates'},inplace=True)
df_dest.drop(columns='word', inplace=True)

df_final = df_dest[['Route','origin','origin_city','origin_coordinates','destination',
                    'destination_city','destination_coordinates']].dropna().copy()

df_final['distance'] = df_final.apply(
    lambda row: calculate_distance(row['origin_coordinates'], 
                                   row['destination_coordinates']), axis=1)

In [ ]:
score_df2=score_df.loc[score_df['score']==100]

In [ ]:
df_new = df[['Route', 'origin', 'destination', 'Overall_Rating', 'Recommended']].copy().dropna()

df_origins = pd.merge(df_new, score_df2[['word','match','coordinates']], left_on= 'origin',right_on='word', how='left')
df_origins.rename(columns={'match':'origin_city','coordinates':'origin_coordinates'},inplace=True)
df_origins.drop(columns='word', inplace=True)

df_dest = pd.merge(df_origins, score_df2[['word','match','coordinates']], left_on= 'destination',right_on='word', how='left')
df_dest.rename(columns={'match':'destination_city','coordinates':'destination_coordinates'},inplace=True)
df_dest.drop(columns='word', inplace=True)

df_final2 = df_dest[['Route','origin','origin_city','origin_coordinates','destination',
                    'destination_city','destination_coordinates']].dropna().copy()

df_final2['distance'] = df_final2.apply(
    lambda row: calculate_distance(row['origin_coordinates'], 
                                   row['destination_coordinates']), axis=1)

In [ ]:
import seaborn as sns
sns.histplot(df_final2['distance'])

In [ ]:
df_final2.loc[df_final2['distance']>17500]
37.7749° N,
122.4194° W

In [ ]:
codes_df.loc[codes_df['coordinates']=='-38.7, -60.016667']